In [1]:
import treeflow.libsbn
import treeflow.model
import numpy as np

newick_file = 'demo-data/dengue.newick'
fasta_file = 'demo-data/dengue.fasta'

inst = treeflow.libsbn.get_instance(newick_file)
q_tree = treeflow.model.construct_tree_approximation(newick_file, inst=inst)

In [2]:
q_tree.sample()

{'topology': {'parent_indices': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([17, 17, 18, 19, 19, 20, 22, 23, 23, 24, 25, 26, 27, 28, 29, 30, 31,
         18, 21, 20, 21, 22, 32, 24, 25, 26, 27, 28, 29, 30, 31, 32])>},
 'heights': <tf.Tensor: shape=(33,), dtype=float32, numpy=
 array([10.00009 , 16.00008 , 16.00009 , 30.00009 , 10.00011 , 38.0001  ,
        31.00009 , 12.00005 , 11.00005 , 13.00006 , 10.00005 ,  8.00005 ,
         0.      , 15.00005 ,  9.00005 , 17.00005 , 18.00005 , 23.016   ,
        26.43827 , 30.754513, 37.13909 , 35.153713, 33.361156, 12.340055,
        13.376332, 14.045527, 14.802625, 14.860653, 15.832997, 17.9914  ,
        18.56426 , 18.726286, 33.327686], dtype=float32)>}

In [3]:
bij = q_tree.model['heights'].distributions[1].bijector.bijectors[0]
init_heights = np.array(bij.tree.node_heights)
to_ratios = q_tree.model['heights'].distributions[1].bijector.bijectors[1]
base = q_tree.model['heights'].distributions[1].distribution
ratio_sample = to_ratios.forward(base.sample())
bij.forward(ratio_sample), init_heights

(<tf.Tensor: shape=(16,), dtype=float32, numpy=
 array([20.252815, 40.39217 , 47.1811  , 68.60814 , 84.813896, 86.27825 ,
        14.002418, 14.195272, 15.632719, 20.261486, 20.363136, 20.842209,
        22.383423, 27.433018, 32.275726, 86.63322 ], dtype=float32)>,
 array([10.        , 16.        , 16.        , 30.        , 10.        ,
        38.        , 31.        , 12.        , 11.        , 13.        ,
        10.        ,  8.        ,  0.        , 15.        ,  9.        ,
        17.        , 18.        , 23.01600077, 26.43827119, 30.75451352,
        37.13909298, 35.15371438, 33.36115766, 12.34005559, 13.37633239,
        14.04552767, 14.80262507, 14.86065326, 15.83299728, 17.99140003,
        18.56426073, 18.72628665, 33.32768631]))

In [4]:
import tensorflow as tf

with tf.GradientTape() as t:
    t.watch(ratio_sample)
    heights = bij.forward(ratio_sample)
    res = tf.reduce_sum(heights * heights)
    
t.gradient(res, ratio_sample)

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([  550.92737 ,  4144.6045  ,  3881.131   , 19220.996   ,
       22935.514   , 23490.318   ,    80.898125,   179.72    ,
         251.0769  ,   457.78253 ,  1072.3301  ,   926.7661  ,
        2899.6716  ,  1639.7069  , 28310.908   ,   530.0461  ],
      dtype=float32)>